In [ ]:
using Pkg;
Pkg.activate("../") 
using statisticalCTMRG
using TensorKit
using KrylovKit
using LinearAlgebra

  Activating project at `c:\Users\psire\work\phd-projects\statisticalCTMRG`


## Step 1: Initialize the boundary tensors

In [115]:
D = 20
V_virt = ℤ₂Space(0 => D, 1 => D)
V_phys = ℤ₂Space(0 => 1, 1 => 1)
trivialspace = one(V_virt)   

# corner tensors
C_ul = randn(V_virt ← V_virt)
C_dr = randn(V_virt ← V_virt)
C_dl = randn(trivialspace ← V_virt ⊗ V_virt)
C_ur = randn(V_virt ⊗ V_virt ← trivialspace)

# edge tensors
Tr_l = randn(V_virt ← V_phys ⊗ V_virt)
Tr_d = randn(V_virt ← V_virt ⊗ V_phys)
Tr_r = randn(V_phys ⊗ V_virt ← V_virt)
Tr_u = randn(V_virt ⊗ V_phys ← V_virt)

env = statisticalCTMRG.envs(C_ul, C_dl, C_dr, C_ur, Tr_l, Tr_d, Tr_r, Tr_u);



## Step 2: Create a projector

### Without truncation

In [116]:
a, b = 2, 1.7
T = generate_local_tensor_Z2(a,b)

@tensor order = (v1, w1, w2, u1) begin Ku[(i,j1);(α,β1)] := env.ul[w1,v1] * env.u[v1,w2,α] * 
                                                            env.l[i,u1,w1] * T[u1,j1,β1,w2] end

@tensor order = (w1, v1, w2, u1) begin Kd[(i,j1);(α1,β)] := env.dl[v1,w1] * env.d[v1,β,w2] * 
                                                            env.l[w1,u1,i] * T[u1,w2,α1,j1] end 

Ku = Ku / norm(Ku, Inf)
Kd = Kd / norm(Kd, Inf)

@tensor L[(β1,β2);(α1,α2)] := Kd[v1,v2,β1,β2]*Ku[v1,v2,α1,α2];

# no truncation
U_L, S_L_full, V_L_T = svd_full(L)
println("No truncation:")
display(S_L_full)
S_L_full_even_sorted = sort(diag(block(S_L_full, Irrep[ℤ₂](0))), rev=true)
S_L_full_odd_sorted = sort(diag(block(S_L_full, Irrep[ℤ₂](1))), rev=true)
@show S_L_full_even_sorted
@show S_L_full_odd_sorted

nothing


No truncation:


80←80 TensorMap{Float64, Rep[ℤ₂], 1, 1, Vector{Float64}}:
 codomain: ⊗(Rep[ℤ₂](0 => 40, 1 => 40))
 domain: ⊗(Rep[ℤ₂](0 => 40, 1 => 40))
 blocks: 
 * Irrep[ℤ₂](0) => 40×40 reshape(view(::Vector{Float64}, 1:1600), 40, 40) with eltype Float64:
 6.20152  0.0     0.0      0.0      …  0.0          0.0         0.0
 0.0      5.4835  0.0      0.0         0.0          0.0         0.0
 0.0      0.0     3.42818  0.0         0.0          0.0         0.0
 0.0      0.0     0.0      3.02195     0.0          0.0         0.0
 0.0      0.0     0.0      0.0         0.0          0.0         0.0
 ⋮                                  ⋱                           
 0.0      0.0     0.0      0.0         0.0          0.0         0.0
 0.0      0.0     0.0      0.0         0.000254697  0.0         0.0
 0.0      0.0     0.0      0.0         0.0          3.03714e-5  0.0
 0.0      0.0     0.0      0.0         0.0          0.0         6.48524e-6

 * Irrep[ℤ₂](1) => 40×40 reshape(view(::Vector{Float64}, 1601:3200), 40, 4

S_L_full_even_sorted = [6.201524474840712, 5.483503931722479, 3.4281752820738745, 3.0219543567679894, 2.5431135206156883, 1.5768984065806224, 1.3608277289717214, 1.0975859981565927, 1.0225147271365491, 0.7529005087765488, 0.7178625773530958, 0.6341690698524712, 0.46799857304343195, 0.4429818442353419, 0.4223499857141397, 0.3094505578292706, 0.2269666526679192, 0.19685839142293907, 0.16307119803799824, 0.13362884247241263, 0.1033998655768211, 0.07937146424245503, 0.07568898797742839, 0.06823456401136932, 0.04932335565101366, 0.03621223479050922, 0.02216605429342018, 0.020236524643607624, 0.016615728061748453, 0.006972689804585301, 0.0061740291518293305, 0.00401024618989913, 0.002988234240250853, 0.0021673899464277314, 0.0012715801824398256, 0.0007401946189495556, 0.000498173676607444, 0.00025469678974547545, 3.0371390699886737e-5, 6.485243468133324e-6]
S_L_full_odd_sorted = [5.652498111137654, 3.8797408672221065, 3.698976940174248, 2.9554886116803725, 2.214158872269865, 2.01277636275213

### With truncation

In [220]:
# Option 1: truncation in each block based on trunc dimension
D_trunc = 10
V_trunc = ℤ₂Space(0 => D_trunc, 1 => D_trunc)
println("-------------Truncation based on dimension:------------")
U_L, S_L, V_L_T = svd_trunc(L, trunc=truncspace(V_trunc)) # -> pick the first 5 singular values in each symmetry sector
display(S_L)

# Option 2: truncation in each block based on norm difference
println("-------------Truncation based on norm difference w.r.t truncated matrix:-------------")
atol = 1.0e-3

U_L, S_L, V_L_T, ϵ_all = svd_trunc(L, trunc=truncerror(; atol = atol)) # dynamically change virtual space s.t. the SVD truncation error is < atol
display(S_L)
@show ϵ_all

U_L_even, S_L_even, V_L_T_even, ϵ_even =  svd_trunc(block(L, Irrep[ℤ₂](0)), trunc=truncerror(; atol = atol))
error_discarded_sv_even = sqrt(norm(block(S_L_full, Irrep[ℤ₂](0)))^2 - norm(S_L_even)^2)
@show abs(ϵ_even - error_discarded_sv_even) < 1e-8

U_L_odd, S_L_odd, V_L_T_odd, ϵ_odd =  svd_trunc(block(L, Irrep[ℤ₂](1)), trunc=truncerror(; atol = atol))
error_discarded_sv_odd = sqrt(norm(block(S_L_full, Irrep[ℤ₂](1)))^2 - norm(S_L_odd)^2)
@show abs(ϵ_odd - error_discarded_sv_odd) < 1e-8

@show ϵ_odd + ϵ_even == ϵ_all

trunc_even_dim = size((block(S_L, Irrep[ℤ₂](0))),1)
L_even = block(L, Irrep[ℤ₂](0))
U_L_trunc_even = block(U_L, Irrep[ℤ₂](0))
S_L_trunc_even = block(S_L, Irrep[ℤ₂](0))
V_L_T_trunc_even = block(V_L_T, Irrep[ℤ₂](0))
@show norm(L_even - U_L_trunc_even * S_L_trunc_even * V_L_T_trunc_even) < atol

trunc_even_odd = size((block(S_L, Irrep[ℤ₂](1))),1)
L_odd = block(L, Irrep[ℤ₂](1))
U_L_trunc_odd = block(U_L, Irrep[ℤ₂](1))
S_L_trunc_odd = block(S_L, Irrep[ℤ₂](1))
V_L_T_trunc_odd = block(V_L_T, Irrep[ℤ₂](1))  
@show norm(L_odd - U_L_trunc_odd * S_L_trunc_odd * V_L_T_trunc_odd) < atol

nothing

-------------Truncation based on dimension:------------


20←20 DiagonalTensorMap{Float64, Rep[ℤ₂], Vector{Float64}}:
 codomain: ⊗(Rep[ℤ₂](0 => 10, 1 => 10))
 domain: ⊗(Rep[ℤ₂](0 => 10, 1 => 10))
 blocks: 
 * Irrep[ℤ₂](0) => 10×10 Diagonal{Float64, SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int64}}, true}}:
 6.20152   ⋅       ⋅        ⋅       …   ⋅        ⋅        ⋅        ⋅ 
  ⋅       5.4835   ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
  ⋅        ⋅      3.42818   ⋅           ⋅        ⋅        ⋅        ⋅ 
  ⋅        ⋅       ⋅       3.02195      ⋅        ⋅        ⋅        ⋅ 
  ⋅        ⋅       ⋅        ⋅           ⋅        ⋅        ⋅        ⋅ 
  ⋅        ⋅       ⋅        ⋅       …   ⋅        ⋅        ⋅        ⋅ 
  ⋅        ⋅       ⋅        ⋅          1.36083   ⋅        ⋅        ⋅ 
  ⋅        ⋅       ⋅        ⋅           ⋅       1.09759   ⋅        ⋅ 
  ⋅        ⋅       ⋅        ⋅           ⋅        ⋅       1.02251   ⋅ 
  ⋅        ⋅       ⋅        ⋅           ⋅        ⋅        ⋅       0.752901

 * Irrep[ℤ₂](1) => 10×10 Diagona

-------------Truncation based on norm difference w.r.t truncated matrix:-------------


72←72 DiagonalTensorMap{Float64, Rep[ℤ₂], Vector{Float64}}:
 codomain: ⊗(Rep[ℤ₂](0 => 36, 1 => 36))
 domain: ⊗(Rep[ℤ₂](0 => 36, 1 => 36))
 blocks: 
 * Irrep[ℤ₂](0) => 36×36 Diagonal{Float64, SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int64}}, true}}:
 6.20152   ⋅       ⋅        ⋅       …   ⋅           ⋅           ⋅ 
  ⋅       5.4835   ⋅        ⋅           ⋅           ⋅           ⋅ 
  ⋅        ⋅      3.42818   ⋅           ⋅           ⋅           ⋅ 
  ⋅        ⋅       ⋅       3.02195      ⋅           ⋅           ⋅ 
  ⋅        ⋅       ⋅        ⋅           ⋅           ⋅           ⋅ 
 ⋮                                  ⋱                          ⋮
  ⋅        ⋅       ⋅        ⋅           ⋅           ⋅           ⋅ 
  ⋅        ⋅       ⋅        ⋅          0.00216739   ⋅           ⋅ 
  ⋅        ⋅       ⋅        ⋅           ⋅          0.00127158   ⋅ 
  ⋅        ⋅       ⋅        ⋅       …   ⋅           ⋅          0.000740195

 * Irrep[ℤ₂](1) => 36×36 Diagonal{Float64, SubArray{Float64, 

ϵ_all = 0.0007368660003708925
abs(ϵ_even - error_discarded_sv_even) < 1.0e-8 = true
abs(ϵ_odd - error_discarded_sv_odd) < 1.0e-8 = true
ϵ_odd + ϵ_even == ϵ_all = true
norm(L_even - U_L_trunc_even * S_L_trunc_even * V_L_T_trunc_even) < atol = true
norm(L_odd - U_L_trunc_odd * S_L_trunc_odd * V_L_T_trunc_odd) < atol = true


In [228]:
function truncate_sector_by_error(aTensor::TensorMap, atol::Real)
    I = sectortype(aTensor)
    T = eltype(aTensor)

    # Dictionaries to hold matrix data for each sector
    U_data = Dict{I, Matrix{T}}()
    S_data = Dict{I, Matrix{T}}()
    V_data = Dict{I, Matrix{T}}()
    
    new_dims = Dict{I, Int}()
    ϵ_total = 0.0

    for (c, b) in blocks(aTensor)
        U, S, V_T, ϵ = svd_trunc(b, trunc=truncerror(; atol = atol))
        
        U_data[c] = U
        S_data[c] = S
        V_data[c] = V_T
        
        new_dims[c] = size(S, 1)
        ϵ_total += ϵ^2
    end

    SpaceType = typeof(space(aTensor, 1))
    V_virt = SpaceType(new_dims...)
    
    U_trunc = TensorMap(U_data, codomain(aTensor), V_virt)    
    S_trunc = TensorMap(S_data, V_virt, V_virt)
    V_trunc = TensorMap(V_data, V_virt, domain(aTensor))
    aTensor_trunc = U_trunc * S_trunc * V_trunc

    return U_trunc, S_trunc, V_trunc, ϵ_total
end

truncate_sector_by_error (generic function with 1 method)

In [229]:
U_trunc, S_trunc, V_trunc, ϵ_total = truncate_sector_by_error(L, 1.0e-3)
display(S_trunc)
@show ϵ_total

nothing

70←70 TensorMap{Float64, Rep[ℤ₂], 1, 1, Vector{Float64}}:
 codomain: ⊗(Rep[ℤ₂](0 => 35, 1 => 35))
 domain: ⊗(Rep[ℤ₂](0 => 35, 1 => 35))
 blocks: 
 * Irrep[ℤ₂](0) => 35×35 reshape(view(::Vector{Float64}, 1:1225), 35, 35) with eltype Float64:
 6.20152  0.0     0.0      0.0      …  0.0         0.0         0.0
 0.0      5.4835  0.0      0.0         0.0         0.0         0.0
 0.0      0.0     3.42818  0.0         0.0         0.0         0.0
 0.0      0.0     0.0      3.02195     0.0         0.0         0.0
 0.0      0.0     0.0      0.0         0.0         0.0         0.0
 ⋮                                  ⋱                          
 0.0      0.0     0.0      0.0         0.0         0.0         0.0
 0.0      0.0     0.0      0.0         0.00298823  0.0         0.0
 0.0      0.0     0.0      0.0         0.0         0.00216739  0.0
 0.0      0.0     0.0      0.0         0.0         0.0         0.00127158

 * Irrep[ℤ₂](1) => 35×35 reshape(view(::Vector{Float64}, 1226:2450), 35, 35) with el

ϵ_total = 0.0007368660003708925
